In [26]:
#from dataset_downloader import dsdl
import dsdl
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
from PIL import Image
import os
import matplotlib.pyplot as plt
import numpy as np
from typing import Any, Callable, Optional, Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils as utils
import torchvision
import torchvision.utils as vutils
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.optim as optim

import time
from torch.optim.optimizer import Optimizer
from typing import List
from torch import Tensor
import numpy


class ARGS():
    def __init__(self):
        self.loss_type = 'CE' # 'CE'/'MARGIN'
        self.smoothness = 2 # 1/2. For margin loss
        self.noise_type = '_' # 'FIINITE'/'GAUSS'
        self.adaptive = False
        self.batch_size=128
        self.depth=16
        self.dropout=0.0
        self.epochs=200
        self.label_smoothing=0.0
        self.learning_rate=0.1
        self.momentum_mode="OFF"
        self.threads=8
        self.rho=0.001
        self.width_factor=8
        self.optimizer='SAM'
        self.momentum=0.9
        self.weight_decay=5e-4
        self.last_layer = "100"
        self.model = "ResNet"
        self.step_lr=1
        self.gpu="cuda:1"
        
args = ARGS()

if args.momentum_mode=="ON":
    args.momentum=0.9
    args.weight_decay=5e-4
elif args.momentum_mode=="OFF":
    args.momentum=0.0
    args.weight_decay=5e-3
else:
    sys.exit(-1)

print("label_smoothing", args.label_smoothing)
print("rho", args.rho)
print("weight_decay",args.weight_decay)
print("learning_rate", args.learning_rate)
print("momentum", args.momentum)
print("noise_type", args.noise_type)
print("threads", args.threads)
print("adaptive", args.adaptive)
print("batch_size", args.batch_size)
print("epochs", args.epochs)
print("optimizer",args.optimizer)
print("last_layer",args.last_layer)
print("model",args.model)
print("gpu",args.gpu)
print("step_lr",args.step_lr)

device = torch.device(args.gpu if torch.cuda.is_available() else "cpu")
print(device)
#ダウンロード
ds = dsdl.load("a9a")

X,y = ds.get_train()
X = X.toarray()
X_te,y_te = ds.get_test()
X_te = X_te.toarray()
X_te = np.block([X_te, np.zeros((16281, 1))])

y = ( (y + 1) / 2 ).astype(np.int32)
y_te = ((y_te + 1)/ 2).astype(np.int32)

X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32) 

X_te = torch.tensor(X_te, dtype=torch.float32) 
y_te = torch.tensor(y_te, dtype=torch.float32)

train_data = torch.utils.data.TensorDataset(X, y)
test_data = torch.utils.data.TensorDataset(X_te, y_te)

data = torch.cat([d[0] for d in DataLoader(train_data)])
mean = data.mean(dim = 0)
std = data.std(dim = 0)
eps =1e-15
X_ = (X - mean)/(std + eps)
X_te_ = (X_te - mean)/(std + eps)

label_smoothing 0.0
rho 0.001
weight_decay 0.005
learning_rate 0.1
momentum 0.0
noise_type FINITE
threads 8
adaptive False
batch_size 128
epochs 200
optimizer SAM
last_layer 100
model ResNet
gpu cuda:1
step_lr 1
cpu


In [5]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
from PIL import Image
import os
import matplotlib.pyplot as plt
import numpy as np
from typing import Any, Callable, Optional, Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils as utils
import torchvision
import torchvision.utils as vutils
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.optim as optim

import time
from torch.optim.optimizer import Optimizer
from typing import List
from torch import Tensor
import numpy

class LibsvmDataset(torch.utils.data.Dataset):
    def __init__(self, data, label, transform=None, target_transform=None):
        self.data = data
        self.label = label
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data = self.data[idx]
        label = self.label[idx]
        if self.transform:
            data = self.transform(data)
        if self.target_transform:
            label = self.target_transform(label)
        return data, label

In [7]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
from PIL import Image
import os
import matplotlib.pyplot as plt
import numpy as np
from typing import Any, Callable, Optional, Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils as utils
import torchvision
import torchvision.utils as vutils
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.optim as optim

import time
from torch.optim.optimizer import Optimizer
from typing import List
from torch import Tensor
import numpy

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(123, 123*5),
            nn.ReLU(),
            nn.Linear(123*5, 1)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [9]:
import torch

class NSAM(torch.optim.Optimizer):
    def __init__(self, params, base_optimizer, rho=0.05, adaptive=False, device = None,noise_type = 'FINITE', **kwargs):
        assert rho >= 0.0, f"Invalid rho, should be non-negative: {rho}"
        noise_list = []

        defaults = dict(rho=rho, adaptive=adaptive, noise_list=noise_list, **kwargs)
        super(NSAM, self).__init__(params, defaults)
        self.base_optimizer = base_optimizer(self.param_groups, **kwargs)
        self.param_groups = self.base_optimizer.param_groups
        self.noise_type = noise_type

        for group in self.param_groups:
            for i,p in enumerate(group['params']):
                group['noise_list'].append(torch.zeros(p.shape, device=device))

    @torch.no_grad()
    def first_step(self, zero_grad=False):
        for group in self.param_groups:
            for i,p in enumerate(group['params']):
                torch.randn(p.shape,out=group['noise_list'][i])
            #print(torch.max(group['noise_list'][0]))

        noise_norm = self._noise_norm()
        for group in self.param_groups:
            scale = group["rho"] / (noise_norm + 1e-12)

            for i,p in enumerate(group["params"]):
                self.state[p]["old_p"] = p.data.clone()
                #e_w = group['noise_list'][i] * scale.to(p)
                e_w = (torch.pow(p, 2) if group["adaptive"] else 1.0) * group['noise_list'][i] * scale.to(p)
                p.add_(e_w) 

        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def second_step(self, zero_grad=False):
        for group in self.param_groups:
            for p in group["params"]:
                if p.grad is None: continue
                p.data = self.state[p]["old_p"]  # get back to "w" from "w + e(w)"

        self.base_optimizer.step()  # do the actual "sharpness-aware" update

        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def step(self, closure=None):
        assert closure is not None, "Sharpness Aware Minimization requires closure, but it was not provided"
        closure = torch.enable_grad()(closure)  # the closure should do a full forward-backward pass

        self.first_step(zero_grad=True)
        closure()
        self.second_step()

    def _noise_norm(self):
        shared_device = self.param_groups[0]["params"][0].device  # put everything on the same device, in case of model parallelism
        if self.noise_type=='FINITE':
            norm = torch.norm( torch.stack( [ ((torch.abs(p) if group["adaptive"] else 1.0) * noise).norm(p=2).to(shared_device) for group in self.param_groups for (p,noise) in zip(group['params'],group['noise_list']) ] ), p=2 )
            return norm
        elif self.noise_type=='GAUSS':
            return torch.scalar_tensor(1.0).to(shared_device)
        ## norm = torch.norm( torch.stack([ ((torch.abs(p) if group["adaptive"] else 1.0) * p.grad).norm(p=2).to(shared_device) for group in self.param_groups for p in group["params"] if p.grad is not None]), p=2)    

    def load_state_dict(self, state_dict):
        super().load_state_dict(state_dict)
        self.base_optimizer.param_groups = self.param_groups

In [10]:
import torch


class SAM(torch.optim.Optimizer):
    def __init__(self, params, base_optimizer, rho=0.05, adaptive=False, **kwargs):
        assert rho >= 0.0, f"Invalid rho, should be non-negative: {rho}"

        defaults = dict(rho=rho, adaptive=adaptive, **kwargs)
        super(SAM, self).__init__(params, defaults)

        self.base_optimizer = base_optimizer(self.param_groups, **kwargs)
        self.param_groups = self.base_optimizer.param_groups

    @torch.no_grad()
    def first_step(self, zero_grad=False):
        grad_norm = self._grad_norm()
        for group in self.param_groups:
            scale = group["rho"] / (grad_norm + 1e-12)

            for p in group["params"]:
                if p.grad is None: continue
                self.state[p]["old_p"] = p.data.clone()
                e_w = (torch.pow(p, 2) if group["adaptive"] else 1.0) * p.grad * scale.to(p)
                p.add_(e_w)  # climb to the local maximum "w + e(w)"

        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def second_step(self, zero_grad=False):
        for group in self.param_groups:
            for p in group["params"]:
                if p.grad is None: continue
                p.data = self.state[p]["old_p"]  # get back to "w" from "w + e(w)"

        self.base_optimizer.step()  # do the actual "sharpness-aware" update

        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def step(self, closure=None):
        assert closure is not None, "Sharpness Aware Minimization requires closure, but it was not provided"
        closure = torch.enable_grad()(closure)  # the closure should do a full forward-backward pass

        self.first_step(zero_grad=True)
        closure()
        self.second_step()

    def _grad_norm(self):
        shared_device = self.param_groups[0]["params"][0].device  # put everything on the same device, in case of model parallelism
        norm = torch.norm(
                    torch.stack([
                        ((torch.abs(p) if group["adaptive"] else 1.0) * p.grad).norm(p=2).to(shared_device)
                        for group in self.param_groups for p in group["params"]
                        if p.grad is not None
                    ]),
                    p=2
               )
        return norm

    def load_state_dict(self, state_dict):
        super().load_state_dict(state_dict)
        self.base_optimizer.param_groups = self.param_groups


In [23]:
def test(epoch,model,test_data, loss_fn, device):
    loss_epoch = 0.0
    acc_epoch = 0.0
    model.eval()
    with torch.no_grad():
        for inputs,labels in test_data:
            labels = labels.to(device)
            inputs = inputs.to(device)
            outputs = model(inputs)
            outputs = torch.reshape(outputs,labels.shape)

            loss = loss_fn(outputs,labels)
            preds = torch.where(outputs < 0.5, 0, 1)

            loss_epoch += loss.item() * inputs.size(0)
            acc_epoch += torch.sum(preds == labels.detach())

        loss_epoch = loss_epoch / len(test_data.dataset)
        acc_epoch = acc_epoch.double() / len(test_data.dataset)
    return loss_epoch,acc_epoch

In [14]:
def train_sgd(epoch,model,train_data,optimizer,loss_fn,device):
    loss_epoch = 0.0
    acc_epoch = 0.0
    model.train()
    for inputs,labels in train_data:
        optimizer.zero_grad()

        labels = labels.to(device)
        inputs = inputs.to(device)
        
        outputs = model(inputs)
        outputs = torch.reshape(outputs,labels.shape)

        loss = loss_fn(outputs,labels)
        #_,preds = torch.max(outputs,1)
        preds = torch.where(outputs < 0.5, 0, 1)

        loss.backward()
        optimizer.step()
        
        loss_epoch += loss.item() * inputs.size(0)
        acc_epoch += torch.sum(preds == labels.detach())

    loss_epoch = loss_epoch / len(train_data.dataset)
    acc_epoch = acc_epoch.double() / len(train_data.dataset)
    return loss_epoch,acc_epoch

def train_nsgd(epoch,model,train_data,optimizer,loss_fn,device):
    loss_epoch = 0.0
    acc_epoch = 0.0
    model.train()
    for inputs,labels in train_data:
        optimizer.zero_grad()

        labels = labels.to(device)
        inputs = inputs.to(device)
        outputs = model(inputs)

        labels = torch.reshape(labels,outputs.shape)

        with torch.no_grad():
            outputs = model(inputs)
            loss = loss_fn(outputs,labels)
            optimizer.first_step()

        loss = loss_fn(model(inputs),labels)
        loss.backward()
        optimizer.second_step(zero_grad=True)
        preds = torch.where(outputs < 0.5, 0, 1)

        with torch.no_grad():
            loss_epoch += loss.item() * inputs.size(0)
            acc_epoch += torch.sum(preds == labels.detach())
            #scheduler(epoch)

    loss_epoch = loss_epoch / len(train_data.dataset)
    acc_epoch = acc_epoch.double() / len(train_data.dataset)
    return loss_epoch,acc_epoch

def train_sam(epoch,model,train_data,optimizer,loss_fn,device):
    loss_epoch = 0.0
    acc_epoch = 0.0
    for inputs,labels in train_data:
        optimizer.zero_grad()

        labels = labels.to(device)
        inputs = inputs.to(device)
        
        #enable_running_stats(model)
        outputs = model(inputs)

        labels = torch.reshape(labels,outputs.shape)

        loss = loss_fn(outputs,labels)
        loss.backward()
        optimizer.first_step(zero_grad=True)

        #disable_running_stats(model)
        loss_fn(model(inputs),labels).backward()
        optimizer.second_step(zero_grad=True)
        preds = torch.where(outputs < 0.5, 0, 1)
        
        with torch.no_grad():
            #scheduler(epoch)
            loss_epoch += loss.item() * inputs.size(0)
            acc_epoch += torch.sum(preds == labels.detach())

    loss_epoch = loss_epoch / len(train_data.dataset)
    acc_epoch = acc_epoch.double() / len(train_data.dataset)
    return loss_epoch,acc_epoch

In [15]:
#データセット作成

train_data = LibsvmDataset(X_, y)
test_data = LibsvmDataset(X_te_, y_te)

train_data = DataLoader(train_data, batch_size=64, shuffle=True)
test_data = DataLoader(test_data, batch_size=64, shuffle=True)



#モデルの作成
model = NeuralNetwork().to(device)
base_optimizer = torch.optim.SGD
loss_fn = torch.nn.BCEWithLogitsLoss()

In [27]:
#学習
if args.optimizer == 'SGD':
    for epoch in range(args.epochs):

        #torch.cuda.synchronize()
        start = time.time()

        if epoch < args.epochs * 4/10:
            lr_ = args.learning_rate
        elif epoch < args.epochs * 6/10:
            lr_ = args.learning_rate * 0.2   
        elif epoch < args.epochs * 8/10:
            lr_ = args.learning_rate* 0.2 ** 2
        else:
            lr_ = args.learning_rate * 0.2 ** 3

        optimizer = torch.optim.SGD(model.parameters(), lr = lr_,momentum = args.momentum, weight_decay = args.weight_decay)
        train_loss,train_acc = train_sgd(epoch,model,train_data,optimizer,loss_fn,device)

        torch.cuda.synchronize()
        train_time = time.time() - start

        print(f"epoch:{epoch}",
                f"learning_rate:{lr_:.4f}",
                f"train_loss:{train_loss:.4f}",
                f"train_acc:{train_acc:.4f}",
                f"time:{train_time:.4f}")

        test_loss,test_acc = test(epoch,model,test_data, loss_fn, device)

        torch.cuda.synchronize()
        test_time = time.time() - start

        print(f"epoch:{epoch}",
                f"test_loss:{test_loss:.4f}",
                f"test_acc:{test_acc:.4f}",
                f"time:{test_time:.4f}")


    print('Finished Training')
elif args.optimizer == 'NOISE_SGD':
    for epoch in range(args.epochs):

        #torch.cuda.synchronize()
        start = time.time()

        if epoch < args.epochs * 4/10:
            lr_ = args.learning_rate
        elif epoch < args.epochs * 6/10:
            lr_ = args.learning_rate * 0.2   
        elif epoch < args.epochs * 8/10:
            lr_ = args.learning_rate* 0.2 ** 2
        else:
            lr_ = args.learning_rate * 0.2 ** 3

        optimizer = NSAM(model.parameters(), base_optimizer, rho=args.rho, adaptive=args.adaptive, device=device, noise_type =  args.noise_type,lr=args.learning_rate, momentum=args.momentum, weight_decay=args.weight_decay)
        train_loss,train_acc = train_nsgd(epoch,model,train_data,optimizer,loss_fn,device)

        torch.cuda.synchronize()
        train_time = time.time() - start

        print(f"epoch:{epoch}",
                f"learning_rate:{lr_:.4f}",
                f"train_loss:{train_loss:.4f}",
                f"train_acc:{train_acc:.4f}",
                f"time:{train_time:.4f}")

        test_loss,test_acc = test(epoch,model,test_data, loss_fn, device)

        torch.cuda.synchronize()
        test_time = time.time() - start

        print(f"epoch:{epoch}",
                f"test_loss:{test_loss:.4f}",
                f"test_acc:{test_acc:.4f}",
                f"time:{test_time:.4f}")


    print('Finished Training')
elif args.optimizer == 'SAM':
    for epoch in range(args.epochs):

        #torch.cuda.synchronize()
        start = time.time()

        if epoch < args.epochs * 4/10:
            lr_ = args.learning_rate
        elif epoch < args.epochs * 6/10:
            lr_ = args.learning_rate * 0.2   
        elif epoch < args.epochs * 8/10:
            lr_ = args.learning_rate* 0.2 ** 2
        else:
            lr_ = args.learning_rate * 0.2 ** 3

        optimizer = SAM(model.parameters(), base_optimizer, rho=args.rho, adaptive=args.adaptive, lr=args.learning_rate, momentum=args.momentum, weight_decay=args.weight_decay)
        train_loss,train_acc = train_sam(epoch,model,train_data,optimizer,loss_fn,device)

        #torch.cuda.synchronize()
        train_time = time.time() - start

        print(f"epoch:{epoch}",
                f"learning_rate:{lr_:.4f}",
                f"train_loss:{train_loss:.4f}",
                f"train_acc:{train_acc:.4f}",
                f"time:{train_time:.4f}")

        test_loss,test_acc = test(epoch,model,test_data, loss_fn, device)

        #torch.cuda.synchronize()
        test_time = time.time() - start

        print(f"epoch:{epoch}",
                f"test_loss:{test_loss:.4f}",
                f"test_acc:{test_acc:.4f}",
                f"time:{test_time:.4f}")


    print('Finished Training') 
else:
    sys.exit(-1)

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)